The final topic in the data preparation series is **Data Cleaning & Preparation - 4. Data Reshaping**.

Data reshaping is the process of restructuring data layouts between 'wide' and 'long' formats to support different analysis and visualization requirements. The 'wide' format presents categories as separate columns (e.g., monthly sales in distinct columns), while the 'long' format stacks these categories into rows with identifying columns (e.g., separate 'Month' and 'Sales' columns).

Based on [`piplist.txt`](./README.md) output, `pandas`, the primary library for these operations (`melt`, `pivot`, `stack`, `unstack`), is available.

**Step 1: Generate Sample Data for Reshaping**

We'll create two dummy DataFrames:
1.  One in a 'wide' format, suitable for converting to 'long' using `melt`.
2.  One in a 'long' format, suitable for converting to 'wide' using `pivot_table`.

In a new Excel cell, enter `=PY` and paste the following code for the **Wide Data** (e.g., quarterly sales), then press **Ctrl+Enter**:

In [ ]:
# Generate dummy Wide Data (e.g., Quarterly Sales)
import pandas as pd
from faker import Faker
import random

fake = Faker()

num_regions = 10
regions = [fake.state() for _ in range(num_regions)]

data = {
    'Region': regions,
    'Q1_Sales_2023': [random.randint(10000, 50000) for _ in range(num_regions)],
    'Q2_Sales_2023': [random.randint(15000, 55000) for _ in range(num_regions)],
    'Q3_Sales_2023': [random.randint(20000, 60000) for _ in range(num_regions)],
    'Q4_Sales_2023': [random.randint(25000, 65000) for _ in range(num_regions)],
    'Q1_Sales_2024': [random.randint(12000, 52000) for _ in range(num_regions)],
    'Q2_Sales_2024': [random.randint(18000, 58000) for _ in range(num_regions)],
}

df_wide = pd.DataFrame(data)

df_wide # Output the DataFrame

Let's assume this data is placed in a range or Table named `QuarterlySalesWide`.

In a **separate, new** Excel cell, enter `=PY` and paste the following code for the **Long Data** (e.g., individual survey responses), then press **Ctrl+Enter**:

In [ ]:
# Generate dummy Long Data (e.g., Survey Responses)
import pandas as pd
from faker import Faker
import random

fake = Faker()

num_respondents = 50
questions = ['Service Quality', 'Product Satisfaction', 'Ease of Use', 'Likelihood to Recommend']
ratings = [1, 2, 3, 4, 5] # Likert scale

data = []
for i in range(num_respondents):
    respondent_id = f'Resp_{100 + i}'
    demographic = random.choice(['Segment A', 'Segment B', 'Segment C'])
    for question in questions:
        rating = random.choice(ratings)
        data.append([respondent_id, demographic, question, rating])

df_long = pd.DataFrame(data, columns=['RespondentID', 'Demographic', 'Question', 'Rating'])

df_long # Output the DataFrame

Let's assume this data is placed in a range or Table named `SurveyResponsesLong`.

**Step 2: Perform Data Reshaping (Melt and Pivot)**

Now, we'll load these two dummy DataFrames from Excel and apply reshaping operations. We will 'melt' the `QuarterlySalesWide` data into a long format and 'pivot' the `SurveyResponsesLong` data into a wide format.

In a **new** Excel cell, enter `=PY` and paste the following code. Replace `"QuarterlySalesWide"` and `"SurveyResponsesLong"` with the actual names of the Excel ranges/Tables where your dummy data is. Press **Ctrl+Enter**.

In [ ]:
# Perform data reshaping using melt and pivot_table
import pandas as pd
import numpy as np

# Load the DataFrames from Excel
# IMPORTANT: Replace the source names with your actual names
df_wide = xl("QuarterlySalesWide[#All]", headers=True)
df_long = xl("SurveyResponsesLong[#All]", headers=True)

# Ensure data types are appropriate
df_wide['Region'] = df_wide['Region'].astype(str)
df_long['RespondentID'] = df_long['RespondentID'].astype(str)
df_long['Demographic'] = df_long['Demographic'].astype(str)
df_long['Question'] = df_long['Question'].astype(str)
df_long['Rating'] = pd.to_numeric(df_long['Rating'], errors='coerce')


# --- Reshaping Method 1: Melt (Wide to Long) ---
# Transform quarterly sales data from columns into rows.
# 'Region' is the identifier variable (stays as a column).
# The sales columns ('Q1_Sales_2023', etc.) will be "unpivoted".
# The column names ('Q1_Sales_2023', etc.) will go into a new column named 'Quarter_Year'.
# The values (sales figures) will go into a new column named 'Sales'.

sales_cols = [col for col in df_wide.columns if 'Sales' in col] # Identify columns to melt

df_long_sales = pd.melt(df_wide,
                        id_vars=['Region'],       # Columns to keep as identifiers
                        value_vars=sales_cols,    # Columns to unpivot
                        var_name='Quarter_Year',  # Name for the new column storing old column names
                        value_name='Sales')       # Name for the new column storing old values

# Optional: Further parse Quarter_Year into separate Quarter and Year columns
# Example: Extract 'Q1', '2023'
df_long_sales[['Quarter', 'Ignore', 'Year']] = df_long_sales['Quarter_Year'].str.split('_', expand=True)
df_long_sales['Year'] = pd.to_numeric(df_long_sales['Year'], errors='coerce') # Convert year to numeric

# Drop the original combined column and the 'Ignore' column if they exist
df_long_sales = df_long_sales.drop(columns=['Quarter_Year', 'Ignore'], errors='ignore')


# --- Reshaping Method 2: Pivot Table (Long to Wide) ---
# Transform survey response data to have questions as columns.
# 'RespondentID' will be the index (rows).
# 'Question' values will become the new columns.
# 'Rating' values will fill the cells.
# If there are duplicate entries for a RespondentID and Question, pivot_table needs an aggregation function.
# Here, we expect one rating per respondent per question, so sum or mean could work, or just rely on default if unique.

df_wide_survey = pd.pivot_table(df_long,
                                index='RespondentID',   # Column to make the index (rows)
                                columns='Question',     # Column whose unique values become new columns
                                values='Rating',        # Column whose values fill the new cells
                                aggfunc='mean')         # How to handle potential duplicate entries (e.g., average rating)

# pivot_table results in a DataFrame with a MultiIndex header for columns if 'columns' has multiple levels.
# Here, 'Question' is a single level, so columns are ['Service Quality', 'Product Satisfaction', ...]
# The index is 'RespondentID'. You might want to reset index to make RespondentID a regular column.
df_wide_survey = df_wide_survey.reset_index()


# Output results
# Return a dictionary containing the reshaped DataFrames
output = {
    'Long Sales Data Head (from Wide)': df_long_sales.head(),
    'Long Sales Data Shape': pd.DataFrame({'Rows': [df_long_sales.shape[0]], 'Columns': [df_long_sales.shape[1]]}),
    'Wide Survey Data Head (from Long)': df_wide_survey.head(),
    'Wide Survey Data Shape': pd.DataFrame({'Rows': [df_wide_survey.shape[0]], 'Columns': [df_wide_survey.shape[1]]})
}

output # Output the dictionary

**Explanation:**

*   We load the `QuarterlySalesWide` and `SurveyResponsesLong` DataFrames using `xl()`. **Remember to replace the source names.**
*   We ensure columns used for merging or values have appropriate data types.
*   `pd.melt()`: This function "unpivots" a DataFrame from a wide to a long format.
    *   `id_vars`: Columns that should remain as identifier variables (e.g., `Region`).
    *   `value_vars`: Columns that contain the values you want to stack (e.g., `Q1_Sales_2023`, etc.).
    *   `var_name`: The name for the new column that will contain the original column names (`Quarter_Year`).
    *   `value_name`: The name for the new column that will contain the values from the original columns (`Sales`).
    *   We also added steps to parse the `Quarter_Year` column into separate `Quarter` and `Year` columns for better structure.
*   `pd.pivot_table()`: This function creates a spreadsheet-style pivot table as a DataFrame. It aggregates data based on specified index and column values.
    *   `index`: The column(s) to form the new DataFrame's index (rows) (e.g., `RespondentID`).
    *   `columns`: The column(s) whose unique values will become the new columns (e.g., `Question`).
    *   `values`: The column(s) whose values will populate the new DataFrame's cells (e.g., `Rating`).
    *   `aggfunc`: The aggregation function to use if there are multiple entries for a given index-column combination (e.g., `mean` to average ratings).
    *   `reset_index()` converts the `RespondentID` index back into a regular column.
*   We return a dictionary containing the heads and shapes of the resulting reshaped DataFrames.

**Viewing the Output:**

*   Click the Python cell, then click the Python icon/button next to the formula bar.
*   Select "Excel Value" (**Ctrl+Shift+Alt+M**) for the DataFrames within the output dictionary to spill them into your sheet. This will show the first few rows and the dimensions of the integrated data.

The reshaped data is now optimized for different analytical purposes: the quarterly sales data in 'long' format facilitates time series analysis and quarter-over-quarter comparisons, while the survey data in 'wide' format is ready for customer segmentation or statistical modeling. The next major category is [**Statistical Analysis**](./04-Statistical%20Analysis_01-Descriptive%20Statistics.md), which builds upon these data preparation techniques to derive meaningful insights from the data.

**Further Analysis:**
* **Multi-Level Reshaping:** Working with hierarchical indices and complex nested data structures using stack/unstack operations
* **Dynamic Reshaping:** Implementing automated reshaping based on data structure detection and business rules
* **Memory-Efficient Reshaping:** Using chunking and iterative processing for large datasets that exceed memory
* **Quality-Aware Reshaping:** Implementing validation checks to ensure data integrity during reshaping operations
* **Cross-Tabulation Analysis:** Creating advanced contingency tables with multiple levels of aggregation